In [1]:
from torchvision.transforms import transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize(size=(224, 224))])
from torchvision import datasets
BASE_PATH="/home/bubbles/Quantbit Internship/casting_data/casting_data/train"
BASE_PATH_TEST="/home/bubbles/Quantbit Internship/casting_data/casting_data/test"
train=datasets.ImageFolder(root=BASE_PATH,transform=transform)
test=datasets.ImageFolder(root=BASE_PATH_TEST,transform=transform)

In [2]:
from torch.utils.data import DataLoader
train_loader=DataLoader(train, batch_size=16,shuffle=True)
test_loader=DataLoader(test,batch_size=16,shuffle=False)
from torchvision import models
model=models.vgg16('IMAGENET1K_V1')

/home/bubbles/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [3]:
for param in model.features.parameters():
    param.requires_grad = False
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
from torch import nn
# model.fc=nn.Linear(1000,2)
# _modules['6']
model.classifier._modules['6']=nn.Linear(4096,2)
# model.classifier._modules['7']=nn.Sigmoid()
from torch.optim import Adam
clf=model.to('cpu')
opt=Adam(clf.parameters(),lr=1e-5)
loss_fn=nn.CrossEntropyLoss()

In [ ]:
import torch
for epoch in range(20):
  total_correct = 0
  total_samples = 0
  for batch in train_loader:
    X,y=batch
    X,y=X.to('cpu'),y.to('cpu')
    yhat=clf(X)
    # print(yhat,y.unsqueeze(1))
    # print(yhat.squeeze(1).float(),y)
    loss=loss_fn(yhat,y)

    opt.zero_grad()
    loss.backward()
    opt.step()
    # print(y,yhat)
    _, predicted = torch.max(yhat, 1)

        # Update the running total of correct predictions and samples
    total_correct += (predicted == y).sum().item()
    total_samples += y.size(0)

    # Calculate the accuracy for this epoch
  accuracy = 100 * total_correct / total_samples
  print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}% Loss:{loss}')


/home/bubbles/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1: Accuracy = 87.35% Loss:0.17238816618919373
